In [47]:
import numpy as np
import pandas as pd
from pyspark.sql.functions import col


In [48]:
beneficiaryDataDf = spark.read.csv('/home/zahra/Documents/HW3/Train_Beneficiarydata-1542865627584.csv', header=True)
inPatientDataDf = spark.read.csv('/home/zahra/Documents/HW3/Train_Inpatientdata-1542865627584.csv', header=True)
outPatientDataDf = spark.read.csv('/home/zahra/Documents/HW3/Train_Outpatientdata-1542865627584.csv', header=True)
potential_fraud = spark.read.csv('/home/zahra/Documents/HW3/Train-1542865627584.csv', header=True)

In [49]:
beneficiaryDataDf.show(n=5,truncate=False,vertical=True)

-RECORD 0-------------------------------------
 BeneID                          | BENE11001  
 DOB                             | 1943-01-01 
 DOD                             | NA         
 Gender                          | 1          
 Race                            | 1          
 RenalDiseaseIndicator           | 0          
 State                           | 39         
 County                          | 230        
 NoOfMonths_PartACov             | 12         
 NoOfMonths_PartBCov             | 12         
 ChronicCond_Alzheimer           | 1          
 ChronicCond_Heartfailure        | 2          
 ChronicCond_KidneyDisease       | 1          
 ChronicCond_Cancer              | 2          
 ChronicCond_ObstrPulmonary      | 2          
 ChronicCond_Depression          | 1          
 ChronicCond_Diabetes            | 1          
 ChronicCond_IschemicHeart       | 1          
 ChronicCond_Osteoporasis        | 2          
 ChronicCond_rheumatoidarthritis | 1          
 ChronicCond_

In [50]:
inPatientDataDf.show(n=5,truncate=False,vertical=True)

-RECORD 0----------------------------
 BeneID                 | BENE11001  
 ClaimID                | CLM46614   
 ClaimStartDt           | 2009-04-12 
 ClaimEndDt             | 2009-04-18 
 Provider               | PRV55912   
 InscClaimAmtReimbursed | 26000      
 AttendingPhysician     | PHY390922  
 OperatingPhysician     | NA         
 OtherPhysician         | NA         
 AdmissionDt            | 2009-04-12 
 ClmAdmitDiagnosisCode  | 7866       
 DeductibleAmtPaid      | 1068       
 DischargeDt            | 2009-04-18 
 DiagnosisGroupCode     | 201        
 ClmDiagnosisCode_1     | 1970       
 ClmDiagnosisCode_2     | 4019       
 ClmDiagnosisCode_3     | 5853       
 ClmDiagnosisCode_4     | 7843       
 ClmDiagnosisCode_5     | 2768       
 ClmDiagnosisCode_6     | 71590      
 ClmDiagnosisCode_7     | 2724       
 ClmDiagnosisCode_8     | 19889      
 ClmDiagnosisCode_9     | 5849       
 ClmDiagnosisCode_10    | NA         
 ClmProcedureCode_1     | NA         
 ClmProcedur

In [51]:
outPatientDataDf.show(n=5,truncate=False,vertical=True)

-RECORD 0----------------------------
 BeneID                 | BENE11002  
 ClaimID                | CLM624349  
 ClaimStartDt           | 2009-10-11 
 ClaimEndDt             | 2009-10-11 
 Provider               | PRV56011   
 InscClaimAmtReimbursed | 30         
 AttendingPhysician     | PHY326117  
 OperatingPhysician     | NA         
 OtherPhysician         | NA         
 ClmDiagnosisCode_1     | 78943      
 ClmDiagnosisCode_2     | V5866      
 ClmDiagnosisCode_3     | V1272      
 ClmDiagnosisCode_4     | NA         
 ClmDiagnosisCode_5     | NA         
 ClmDiagnosisCode_6     | NA         
 ClmDiagnosisCode_7     | NA         
 ClmDiagnosisCode_8     | NA         
 ClmDiagnosisCode_9     | NA         
 ClmDiagnosisCode_10    | NA         
 ClmProcedureCode_1     | NA         
 ClmProcedureCode_2     | NA         
 ClmProcedureCode_3     | NA         
 ClmProcedureCode_4     | NA         
 ClmProcedureCode_5     | NA         
 ClmProcedureCode_6     | NA         
 DeductibleA

In [52]:
allDataDf = inPatientDataDf.drop(*['AdmissionDt', 'DiagnosisGroupCode', 'DischargeDt']).union(outPatientDataDf)

In [53]:
allDataDf.show(n=5,truncate=False,vertical=True)

-RECORD 0----------------------------
 BeneID                 | BENE11001  
 ClaimID                | CLM46614   
 ClaimStartDt           | 2009-04-12 
 ClaimEndDt             | 2009-04-18 
 Provider               | PRV55912   
 InscClaimAmtReimbursed | 26000      
 AttendingPhysician     | PHY390922  
 OperatingPhysician     | NA         
 OtherPhysician         | NA         
 ClmAdmitDiagnosisCode  | 7866       
 DeductibleAmtPaid      | 1068       
 ClmDiagnosisCode_1     | 1970       
 ClmDiagnosisCode_2     | 4019       
 ClmDiagnosisCode_3     | 5853       
 ClmDiagnosisCode_4     | 7843       
 ClmDiagnosisCode_5     | 2768       
 ClmDiagnosisCode_6     | 71590      
 ClmDiagnosisCode_7     | 2724       
 ClmDiagnosisCode_8     | 19889      
 ClmDiagnosisCode_9     | 5849       
 ClmDiagnosisCode_10    | NA         
 ClmProcedureCode_1     | NA         
 ClmProcedureCode_2     | NA         
 ClmProcedureCode_3     | NA         
 ClmProcedureCode_4     | NA         
 ClmProcedur

In [54]:
new_df = allDataDf.join(potential_fraud, on='Provider').select(col('Provider'), col("PotentialFraud"), col("AttendingPhysician"))

In [55]:
new_df.show()

+--------+--------------+------------------+
|Provider|PotentialFraud|AttendingPhysician|
+--------+--------------+------------------+
|PRV55912|           Yes|         PHY390922|
|PRV55907|            No|         PHY318495|
|PRV56046|            No|         PHY372395|
|PRV52405|            No|         PHY369659|
|PRV56614|            No|         PHY379376|
|PRV54986|           Yes|         PHY402711|
|PRV54090|            No|         PHY412314|
|PRV51148|           Yes|         PHY346286|
|PRV55839|            No|         PHY385030|
|PRV55215|           Yes|         PHY355604|
|PRV55193|           Yes|         PHY397979|
|PRV51145|           Yes|         PHY329774|
|PRV55846|            No|         PHY363584|
|PRV52283|           Yes|         PHY382905|
|PRV52283|           Yes|         PHY432797|
|PRV56588|           Yes|         PHY401860|
|PRV56118|            No|         PHY377952|
|PRV57191|           Yes|         PHY398500|
|PRV54675|           Yes|         PHY432444|
|PRV54676|

In [56]:
from graphframes import *

In [57]:
from pyspark.ml.feature import VectorAssembler

In [58]:
edges = new_df.select(*['Provider', 'AttendingPhysician'])\
        .withColumnRenamed('Provider', 'src')\
        .withColumnRenamed('AttendingPhysician', 'dst')

In [59]:
verticies = edges.select('src').distinct()\
            .union(edges.select('dst').distinct())\
            .withColumnRenamed('src', 'id')

In [60]:
g = GraphFrame(verticies, edges)

In [61]:
from igraph import *

In [62]:
ig = Graph.TupleList(g.edges.collect(), directed=True)
out = plot(ig, vertex_size=1, bbox=(0, 0, 1000, 1000))
out.save('./graph2.png')

In [63]:
v = g.vertices["id"]["PRV52405"]

In [64]:
from pyspark.sql.functions import st

ImportError: cannot import name 'st' from 'pyspark.sql.functions' (/opt/spark/python/pyspark/sql/functions.py)

In [65]:
degs = g.degrees

In [66]:
provider_degrees = degs.filter(degs.id.startswith('PRV'))
physians_degrees = degs.filter(degs.id.startswith('PHY'))

In [68]:
communities = g.labelPropagation(maxIter=5)
communities.persist().show(10)

22/02/11 17:27:52 WARN CacheManager: Asked to cache already cached data.


+---------+-------------+
|       id|        label|
+---------+-------------+
|PHY320544| 146028888481|
|PHY320993|1640677507497|
|PHY335236|1374389535120|
|PHY430800|1202590843288|
|PHY402660| 592705487314|
|PHY322314| 532575945093|
|PHY401366| 463856468394|
|PHY432893|1365799600522|
|PHY336438|1297080123879|
|PHY324524| 214748365235|
+---------+-------------+
only showing top 10 rows



In [72]:
from graphframes.examples import Graphs

# Run PageRank until convergence to tolerance "tol".
results = g.pageRank(resetProbability=0.15, tol=0.01)
# Display resulting pageranks and final edge weights
# Note that the displayed pagerank may be truncated, e.g., missing the E notation.
# In Spark 1.5+, you can use show(truncate=False) to avoid truncation.
results.vertices.select("id", "pagerank").show()
results.edges.select("src", "dst", "weight").show()

+---------+------------------+
|       id|          pagerank|
+---------+------------------+
|PHY320544|0.9513623735200238|
|PHY320993|0.9516772436074642|
|PHY335236|0.9892434005622412|
|PHY430800| 0.960408833339948|
|PHY402660|0.9595562192837173|
|PHY322314|0.9572020990576643|
|PHY401366|0.9622912280095485|
|PHY432893|1.1646978679241307|
|PHY336438| 0.955299476379604|
|PHY324524|0.9904330283200745|
|PHY327091|0.9557829485667813|
|PHY402294|1.0977777321556657|
| PRV55432|0.9500556626571458|
|PHY320753|0.9768547991590896|
|PHY417041|0.9811151747055525|
|PHY400231|0.9541969309302667|
| PRV57301|0.9500556626571458|
|PHY335112|0.9925581528286497|
|PHY353562|0.9513436169047672|
|PHY422584|0.9543283997643339|
+---------+------------------+
only showing top 20 rows



+--------+---------+--------------------+
|     src|      dst|              weight|
+--------+---------+--------------------+
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY379811|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV54888|PHY434094|0.002421307506053269|
|PRV55039|PHY402167|4.859086491739552...|
|PRV55039|PHY402167|4.859086491739552...|
+--------+---------+--------------

In [74]:
 df = new_df.join(results)

AttributeError: 'GraphFrame' object has no attribute '_jdf'

In [ ]:
shortestPaths = g.shortestPaths(landmarks=[x.id for x in g.vertices.collect()])
pathLength = shortestPaths.select('id', explode('distances'))
distances = pathLength.groupBy('key')
sumOfGroupedDistances = distances.agg(functions.sum('value').alias('c'))